# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

In [71]:
import pandas as pd
import numpy as np
from scipy import stats

In [72]:
data = pd.io.stata.read_stata('1538005773_dsc_racial_disc/EDA_racial_discrimination/data/us_job_market_discrimination.dta')

In [73]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [74]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1,0,1,0,0,0,0,0,0,
1,b,1,3,3,6,0,1,1,0,316,...,1,0,1,0,0,0,0,0,0,
2,b,1,4,1,6,0,0,0,0,19,...,1,0,1,0,0,0,0,0,0,
3,b,1,3,4,6,0,1,0,1,313,...,1,0,1,0,0,0,0,0,0,
4,b,1,3,3,22,0,0,0,0,313,...,1,1,0,0,0,0,0,1,0,Nonprofit


In [75]:
w = data[data.race=='w']
b = data[data.race=='b']

# Question 1

What test is appropriate for this problem? Does CLT apply?

A test to determine difference in population proportion of calls received between the two groups: black sounding names and white sounding names.

# Question 2

What are the null and alternate hypotheses?

Let's define

pb = proportion of resumes with black sounding names that got a call

pw = proportion of resumes with white sounding names that got a call

Then the null and alternate hypothesis are:

H0: pb = pw

Ha: pb < pw

alpha = 0.05

In [76]:
# Explore the data
w.columns

Index([u'id', u'ad', u'education', u'ofjobs', u'yearsexp', u'honors',
       u'volunteer', u'military', u'empholes', u'occupspecific', u'occupbroad',
       u'workinschool', u'email', u'computerskills', u'specialskills',
       u'firstname', u'sex', u'race', u'h', u'l', u'call', u'city', u'kind',
       u'adid', u'fracblack', u'fracwhite', u'lmedhhinc', u'fracdropout',
       u'fraccolp', u'linc', u'col', u'expminreq', u'schoolreq', u'eoe',
       u'parent_sales', u'parent_emp', u'branch_sales', u'branch_emp', u'fed',
       u'fracblack_empzip', u'fracwhite_empzip', u'lmedhhinc_empzip',
       u'fracdropout_empzip', u'fraccolp_empzip', u'linc_empzip', u'manager',
       u'supervisor', u'secretary', u'offsupport', u'salesrep', u'retailsales',
       u'req', u'expreq', u'comreq', u'educreq', u'compreq', u'orgreq',
       u'manuf', u'transcom', u'bankreal', u'trade', u'busservice',
       u'othservice', u'missind', u'ownership'],
      dtype='object')

In [77]:
len(b.race)

2435

In [78]:
len(w.race)

2435

In [79]:
#data.info()

print("Number of black resumes that received call",sum(b.call))

print("Number of white resumes that received call",sum(w.call))

('Number of black resumes that received call', 157.0)
('Number of white resumes that received call', 235.0)


# Question 3

Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.

### Using permutation approach

Define relevant functions. Note that taking mean of binomial distribution, such as resumes receiving call or not, gives the population proportion.

In [80]:
def permutation_sample(data1, data2):
    """Generate a permutation sample from two data sets."""

    # Concatenate the data sets: data
    data = np.concatenate((data1, data2))

    # Permute the concatenated array: permuted_data
    permuted_data = np.random.permutation(data)

    # Split the permuted array into two: perm_sample_1, perm_sample_2
    perm_sample_1 = permuted_data[:len(data1)]
    perm_sample_2 = permuted_data[len(data1):]

    return perm_sample_1, perm_sample_2

In [81]:
#define function that gives required number of permutation replicates

def draw_perm_reps(data_1, data_2, func, size=1):
    """Generate multiple permutation replicates."""

    # Initialize array of replicates: perm_replicates
    perm_replicates = np.empty(size)

    for i in range(size):
        # Generate permutation sample
        perm_sample_1, perm_sample_2 = permutation_sample(data_1, data_2)

        # Compute the test statistic
        perm_replicates[i] = func(perm_sample_1, perm_sample_2)

    return perm_replicates
    
    

In [82]:
def diff_of_means(data_1, data_2):
    """Difference in means of two arrays."""

    # The difference of means of data_1, data_2: diff
    diff = np.mean(data_1) - np.mean(data_2)

    return diff

In [83]:
#get replicates and compute statistic for hypothesis test

# Compute difference in population proportion: empirical_diff_means
empirical_diff_means = diff_of_means(w.call, b.call)

# Draw 10,000 permutation replicates: perm_replicates
perm_replicates = draw_perm_reps(w.call, b.call,
                                 diff_of_means, size=100000)

# Compute p-value: p
p = np.sum(perm_replicates >= empirical_diff_means) / float(len(perm_replicates))

# Print the result
print('p-value =', p)

('p-value =', 2.0000000000000002e-05)


In [84]:
# Compute p-value: p
p = np.sum(perm_replicates >= empirical_diff_means) / float(len(perm_replicates))

# Print the result
print('p-value =', p)

('p-value =', 2.0000000000000002e-05)


95% confidence level

In [85]:
np.percentile(perm_replicates, [2.5, 97.5])

array([-0.01560576,  0.01478439])

In [86]:
empirical_diff_means

0.032032855

In [87]:
estimate_proportion_white = np.mean(w.call)
print(estimate_proportion_white)

estimate_proportion_black = np.mean(b.call)
print(estimate_proportion_black)

0.0965092
0.0644764


### Frequentist approach

In [88]:
standard_error_white = np.sqrt( (estimate_proportion_white) * (1 - estimate_proportion_white)/(len(w.call)))

standard_error_black = np.sqrt( (estimate_proportion_black) * (1 - estimate_proportion_black)/(len(b.call)))

standard_error_diff_of_mean = np.sqrt( standard_error_white**2 + standard_error_black**2)

z_statistic = empirical_diff_means /standard_error_diff_of_mean
print("z-statistic", z_statistic)

p_value = stats.norm.sf(z_statistic)
p_value

('z-statistic', 4.1155505190022987)


1.9312819064548448e-05

95% confidence interval

In [89]:
z_statistic_95CI = stats.norm.ppf(0.975)

margin_of_error = z_statistic_95CI*standard_error_diff_of_mean

print("margin of error", margin_of_error)

confidence_interval = [-1*margin_of_error, +1*margin_of_error]
confidence_interval

('margin of error', 0.015255126027012737)


[-0.015255126027012737, 0.015255126027012737]

# Question 4

Write a story describing the statistical significance in the context or the original problem.

Since p-value << alpha, we can reject the null hypothesis. The proportion of calls received for resumes with black sounding names is lower than the proportion of calls received for resumes with white sounding names in a statistically significant manner.

# Question 5

Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

The above hypothesis test on shows that race is an important factor. Black resumes receive 3% lower calls compared to white resumes. Therefore, I assess this is also practically significant.

However, it doesn't mean that race is the most important factor. We would have to look at other factors such as education, work experience etc to determine the importance of these other factors. I am looking forward to other techniques in the course such as Principal Component Analysis and Machine Learning Algorithms (Lasso/Ridge regression) to determine how the different factors contribute to the likelihood of receiving calls. 
